In [1]:
print("Ok")

Ok


In [2]:
%pwd

'e:\\LangChain\\Doctor Chatbot\\GenAI-Medical-Chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd


'e:\\LangChain\\Doctor Chatbot\\GenAI-Medical-Chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents 


In [7]:
extracted_data = load_pdf_file(data= "Data/")

In [8]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("Length of Text chunks ",len(text_chunks))

Length of Text chunks  40000


In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
def download_hugging_face_embeddings():
    os.environ['HF_HOME'] = "E:/hf_cache"
    os.environ['TRANSFORMERS_CACHE'] = "E:/hf_cache/transformers"
    embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

C:\Users\debna\AppData\Local\Temp\ipykernel_19232\1593664868.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
e:\LangChain\Doctor Chatbot\GenAI-Medical-Chatbot\myenv\Lib\site-packages\transformers\utils\hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
e:\LangChain\Doctor Chatbot\GenAI-Medical-Chatbot\myenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated f

In [13]:
query_result = embeddings.embed_query("hello")
print("Length",len(query_result))

Length 384


In [82]:
import os
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')


In [18]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os 

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medicalbot"

pc.create_index(
    name= index_name,
    dimension= 384,
    metric='cosine',
    spec=ServerlessSpec(
        cloud="aws",
        region='us-east-1'
    )
)

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-73ep475.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [19]:
import os 
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [20]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks ,
    index_name = index_name,
    embedding= embeddings
)

In [21]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding= embeddings
)

In [22]:
docsearch

In [79]:
retriever = docsearch.as_retriever(search_type = "similarity",search_kwargs = {'k':5})

In [80]:
retrieved_docs = retriever.invoke("What is Acne?")

In [81]:
retrieved_docs

[Document(id='92a6ef13-54e6-4dc5-b317-8926546592ca', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 55.0, 'page_label': '26', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data\\The-Gale-Encyclopedia-of-Medicine-3rd-Edition-staibabussalamsula.ac_.id_.pdf', 'total_pages': 4505.0}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne'),
 Document(id='ac48bfac-9575-4fb1-87e1-f1fdfdbc8010', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 55.0, 'page_label': '26', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data\\The-Gale-Encyclopedia-of-Medicine-3rd-Edition-staibabussalamsula.ac_.id_.pdf', 'total_pages': 4505.0}, page_content='Sebaceous follicles— A structure found within the\nskin that houses the oil-producing glands and hair\nfollicles, where pimples form.\nS

In [103]:
from langchain_community.chat_models import ChatOllama  # for LangChain >= 0.1.14

llm = ChatOllama(
    model="llama3",  # or any other local model you've pulled (e.g., llama3, mistral, etc.)
    temperature=0.4,
    max_tokens=500
)


In [104]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    """
You are a highly knowledgeable and trustworthy medical assistant.
Use the following CONTEXT from the Gale Encyclopedia of Medicine (3rd Edition) to answer the user's question.
Be factual, concise, and avoid speculation.
If the answer cannot be found in the context, reply with "I'm sorry, I do not have information on that topic."

Use bullet points or short paragraphs where helpful. Mention the source context only if relevant.
\n\n
{context}

"""
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system" ,system_prompt),
        ("human","{input}")
    ]
)

In [105]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [106]:
response = rag_chain.invoke({"input" : "What is Cancer?"})
print(response['answer'])

According to the Gale Encyclopedia of Medicine (3rd Edition), cancer is not just one disease but a large group of almost 100 diseases. The two main characteristics of cancer are:

* Uncontrolled growth of cells in the human body
* The ability of these cells to migrate from the original site and spread to distant sites

If the spread is not controlled, cancer can result in death.
